In [1]:
using HDF5
using FastGaussQuadrature
using LaTeXStrings
using LinearAlgebra
using Particles
using Random
using ReducedBasisMethods
using SparseArrays
using Statistics

In [2]:
fpath = "../runs/BoT_Np5e4_k_010_050_np_10_T25_projections.h5"

params = read_sampling_parameters(fpath)

μₜᵣₐᵢₙ = h5read(fpath, "parameters/mu_train")

IP = IntegratorParameters(fpath)

poisson = PoissonSolverPBSplines(fpath)

Ψ = h5read(fpath, "projections/Psi");
Ψₑ = h5read(fpath, "projections/Psi_e");
Πₑ = sparse(h5read(fpath, "projections/Pi_e"));

In [3]:
# Reference draw
P₀ = ParticleList(h5read(fpath, "initial_condition/x_0"),
                  h5read(fpath, "initial_condition/v_0"),
                  h5read(fpath, "initial_condition/w") );

In [4]:
params

(κ = 0.3, ε = 0.03, a = 0.1, v₀ = 4.5, σ = 0.5, χ = 1.0)

# Test

In [5]:
μₜᵣₐᵢₙ

10×5 Matrix{Float64}:
 0.1       0.03  0.1  4.5  0.5
 0.144444  0.03  0.1  4.5  0.5
 0.188889  0.03  0.1  4.5  0.5
 0.233333  0.03  0.1  4.5  0.5
 0.277778  0.03  0.1  4.5  0.5
 0.322222  0.03  0.1  4.5  0.5
 0.366667  0.03  0.1  4.5  0.5
 0.411111  0.03  0.1  4.5  0.5
 0.455556  0.03  0.1  4.5  0.5
 0.5       0.03  0.1  4.5  0.5

In [6]:
nₜₑₛₜ = 10
κₜₑₛₜ_ₘᵢₙ = 0.1; κₜₑₛₜ_ₘₐₓ = 0.5

μₜₑₛₜ = zeros(nₜₑₛₜ, 5)
for i in 1:nₜₑₛₜ
    μₜₑₛₜ[i,:] = [κₜₑₛₜ_ₘᵢₙ, params.ε, params.a, params.v₀, params.σ]
end

λ = 0
for i in 1:nₜₑₛₜ
    if nₜₑₛₜ > 1
        μₜₑₛₜ[i,1] = rand(1)[1]*(κₜₑₛₜ_ₘₐₓ - κₜₑₛₜ_ₘᵢₙ) + κₜₑₛₜ_ₘᵢₙ
#         μₜₑₛₜ[i,1] = (1-λ)*κₜₑₛₜ_ₘᵢₙ + λ*κₜₑₛₜ_ₘₐₓ
#         λ += 1/(nₜₑₛₜ-1)
    end
end  

μₜₑₛₜ = μₜₑₛₜ[sortperm(μₜₑₛₜ[:, 1]), :]

10×5 Matrix{Float64}:
 0.106467  0.03  0.1  4.5  0.5
 0.207656  0.03  0.1  4.5  0.5
 0.241853  0.03  0.1  4.5  0.5
 0.255618  0.03  0.1  4.5  0.5
 0.271879  0.03  0.1  4.5  0.5
 0.289805  0.03  0.1  4.5  0.5
 0.3057    0.03  0.1  4.5  0.5
 0.397155  0.03  0.1  4.5  0.5
 0.413984  0.03  0.1  4.5  0.5
 0.451935  0.03  0.1  4.5  0.5

In [7]:
GC.gc()

In [8]:
IPₜₑₛₜ = IntegratorParameters(IP.dt, IP.nₜ, IP.nₜ+1, IP.nₕ, IP.nₚ, nₜₑₛₜ)
ICₜₑₛₜ = IntegratorCache(IPₜₑₛₜ);

In [9]:
@time Rₜₑₛₜ = ReducedBasisMethods.integrate_vp(P₀, μₜₑₛₜ, params, poisson, IPₜₑₛₜ, ICₜₑₛₜ;
                                              given_phi = false, save = true);
# Xₜₑₛₜ = Rₜₑₛₜ.X
# Vₜₑₛₜ = Rₜₑₛₜ.V
# Φₜₑₛₜ = Rₜₑₛₜ.Φ;

parameter nb. 1


parameter nb. 2


parameter nb. 3


parameter nb. 4


parameter nb. 5


...

parameter nb. 7


parameter nb. 8


parameter nb. 9


parameter nb. 10


 12.094229 seconds (3.16 M allocations: 178.740 MiB, 0.37% gc time, 14.05% compilation time)


In [10]:
Φₜₑₛₜ = copy(Rₜₑₛₜ.Φ)
size(Φₜₑₛₜ)

(16, 2510)

In [11]:
# no saving
@time ReducedBasisMethods.integrate_vp(P₀, μₜₑₛₜ, params, poisson, IPₜₑₛₜ, ICₜₑₛₜ;
                                        given_phi = false, save = false);

parameter nb. 1


parameter nb. 2


parameter nb. 3


parameter nb. 4


parameter nb. 5


...

parameter nb. 7


parameter nb. 8


parameter nb. 9


parameter nb. 10


  9.667537 seconds (15.74 k allocations: 6.491 MiB)


# Reduced Model

In [12]:
k = size(Ψ)[2]
kₑ = size(Ψₑ)[2]
k, kₑ

(131, 637)

In [13]:
RIC = ReducedIntegratorCache(IPₜₑₛₜ, k, kₑ);

In [14]:
ΨᵀPₑ = Ψ' * Ψₑ * inv(Πₑ' * Ψₑ)
ΠₑᵀΨ = Πₑ' * Ψ;

In [15]:
@time Rᵣₘ = reduced_integrate_vp(P₀, Ψ, ΨᵀPₑ, ΠₑᵀΨ, μₜₑₛₜ, params, poisson, IPₜₑₛₜ, RIC;
                                   DEIM=true, given_phi = false, save = true);
# Xᵣₘ = Ψ * Rᵣₘ.Zₓ
# Vᵣₘ = Ψ * Rᵣₘ.Zᵥ
# Φᵣₘ = Rᵣₘ.Φ;

running parameter nb. 1 with chi = 0.35489090561437864
running parameter nb. 

2 with chi = 0.6921868648106393


running parameter nb. 3 with chi = 0.8061750353789653
running parameter nb. 

4 with chi = 0.8520608984337229


running parameter nb. 5 with chi = 0.9062638133319975
running parameter nb. 

...

running parameter nb. 7 with chi = 1.0190002850029842
running parameter nb. 

8 with chi = 1.3238488450572565


running parameter nb. 9 with chi = 1.3799473384249095
running parameter nb. 

10 with chi = 1.506449277634938


  9.361020 seconds (1.82 M allocations: 1.781 GiB, 3.43% gc time, 11.64% compilation time)


In [16]:
# no saving
@time reduced_integrate_vp(P₀, Ψ, ΨᵀPₑ, ΠₑᵀΨ, μₜₑₛₜ, params, poisson, IPₜₑₛₜ, RIC;
                            DEIM=true, given_phi = false, save=false);

running parameter nb. 1 with chi = 0.35489090561437864
running parameter nb. 

2 with chi = 0.6921868648106393


running parameter nb. 3 with chi = 0.8061750353789653
running parameter nb. 

4 with chi = 0.8520608984337229


running parameter nb. 5 with chi = 0.9062638133319975
running parameter nb. 

...

running parameter nb. 7 with chi = 1.0190002850029842
running parameter nb. 

8 with chi = 1.3238488450572565


running parameter nb. 9 with chi = 1.3799473384249095
running parameter nb. 

10 with chi = 1.506449277634938


  8.886311 seconds (33.78 k allocations: 1.681 GiB, 1.86% gc time)


In [17]:
# Saving
h5save("../runs/BoT_Np5e4_k_010_050_np_10_T25_DEIM.h5", IPₜₑₛₜ, poisson, params, μₜᵣₐᵢₙ, μₜₑₛₜ, Rₜₑₛₜ, Rᵣₘ, Ψ);

In [18]:
norm(Rᵣₘ.Φ - Φₜₑₛₜ)

64.13801036761036

In [19]:
norm(Rₜₑₛₜ.Φ - Φₜₑₛₜ)

0.0